In [ ]:
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
import senteval


In [ ]:
tokenizer_path = "bert-base-uncased"  

tokenizer = BertTokenizer.from_pretrained(tokenizer_path, do_lower_case=True)

In [ ]:
import numpy as np
import senteval
def prepare(params, samples):
    return 

def batcher(_, batch):
    batch = [" ".join(sts) for sts in batch]
    inputs = tokenizer(batch, return_tensors="pt",
            add_special_tokens=True,
            pad_to_max_length=True,         # Pad sentence to max length
            truncation=True)
    for k in inputs:
        inputs[k] = inputs[k].to("cuda")
    res = model(**inputs)["hidden_states"]
    res = res[-1][:, 0].detach().cpu().numpy()
    return res


models = ["bert-base-uncased"] + [f"./finetuned-scrumbled-wikitext2-3e-4/checkpoint-{i}" for i in range(5520 + 80, 40000, 80)]
device='cuda'

for model_name in models:
    model = BertForSequenceClassification.from_pretrained(model_name, output_hidden_states=True)
    model = model.to(device)

    model = model.to(device)
    PATH_TO_DATA = "SentEval/data"
    PARAMS = {'task_path': PATH_TO_DATA, 'usepytorch': False, 'kfold': 3, 'batch_size': 8}
    PARAMS['classifier'] = {'max_iter': 10000, 'nhid': 768}
    transfer_tasks = ['CR', 'MR', 'MPQA', 'SUBJ', 'SST2', 'SST5', 'Depth', 'Tense', 'Length']
    import warnings
    def fxn():
        warnings.warn("deprecated", DeprecationWarning)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        se = senteval.engine.SE(PARAMS, batcher, prepare)
        results = se.eval(transfer_tasks)
    with open("probing_results.csv", "a") as fout:
        print(model_name, results, sep="\t", file=fout)